https://github.com/urbanlist/sunset-list/blob/master/analysis_picture.ipynb

https://github.com/robintw/Breathing-Spaces-code/blob/master/Azure%20Table%20Playground.ipynb

In [1]:
import yaml
with open("config.yml", 'r') as ymlfile:
    cfg = yaml.load(ymlfile)

for section in cfg:
    print(section)
#print(cfg['azureblob'])

azureblob


/home/redne/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
from azure.storage.blob import BlockBlobService as BlobService
from azure.cosmosdb.table.tableservice import TableService
from azure.cosmosdb.table.models import Entity

TABLENAME = cfg['azureblob']['tablename']

table_service = TableService(connection_string=cfg['azureblob']['connectionKey'])
entities = table_service.query_entities(TABLENAME, filter=cfg['azureblob']['partitionConfig'])

In [6]:
entity = entities.items[200]
entity

{'PartitionKey': 'PoeRecycle_test',
 'RowKey': '1178',
 'Timestamp': datetime.datetime(2020, 1, 8, 23, 40, 29, 80830, tzinfo=tzlocal()),
 'features': '[0.7311899  0.6004813  0.05073603 ... 0.02860925 0.51185036 0.0799128 ]',
 'image': '<PIL.Image.Image image mode=RGB size=224x224 at 0x7F63DB547690>',
 'image_folder': 'aHR0cHM6Ly9idHBvY3ZoZHMuYmxvYi5jb3JlLndpbmRvd3MubmV0L3BvZG9jcy9CUlpNU0ludi85NzA3ODc1OS8wMDk3MDc4NzU5LzFfUE9FX01JQ1JPX1hCT1hfT1NJUklTJkdVQVJESUFOU19GWTE2X0RlY2VtYmVyLnBkZg2',
 'image_path': './mnt/poeBlob1/output_images/aHR0cHM6Ly9idHBvY3ZoZHMuYmxvYi5jb3JlLndpbmRvd3MubmV0L3BvZG9jcy9CUlpNU0ludi85NzA3ODc1OS8wMDk3MDc4NzU5LzFfUE9FX01JQ1JPX1hCT1hfT1NJUklTJkdVQVJESUFOU19GWTE2X0RlY2VtYmVyLnBkZg2/normalized_images_82.jpg',
 'image_path_code': <azure.cosmosdb.table.models.EntityProperty at 0x7f789382ec90>,
 'etag': 'W/"datetime\'2020-01-08T23%3A40%3A29.0808301Z\'"'}

### Explore data in Azure blob storage with pandas

https://docs.microsoft.com/en-us/azure/machine-learning/team-data-science-process/explore-data-blob

https://www.jitsejan.com/creating-dataframe-from-table-storage.html

In [42]:
import pandas as pd
from azure.cosmosdb.table.tableservice import TableService

CONNECTION_STRING = cfg['azureblob']['connectionKey']
SOURCE_TABLE = cfg['azureblob']['tablename']

def set_table_service():
    """ Set the Azure Table Storage service """
    return TableService(connection_string=CONNECTION_STRING)

def get_dataframe_from_table_storage_table(table_service, filter_query):
    """ Create a dataframe from table storage data """
    return pd.DataFrame(get_data_from_table_storage_table(table_service,
                                                          filter_query))

def get_data_from_table_storage_table(table_service, filter_query):
    """ Retrieve data from Table Storage """
    for record in table_service.query_entities(
        SOURCE_TABLE, filter=filter_query
    ):
        yield record

In [48]:
#fq = "PartitionKey eq 'PoeRecycle_test'"
fq = cfg['azureblob']['partitionConfig']
ts = set_table_service()
df = get_dataframe_from_table_storage_table(table_service=ts,
                                            filter_query=fq)

In [49]:
df.head()

,PartitionKey,RowKey,Timestamp,features,image,image_folder,image_path,image_path_code,etag
0,PoeRecycle_test,0,2020-01-08 23:40:19.977464+00:00,[1.0852401 0.5258224 0.00315639 ... 0.372240...,<PIL.Image.Image image mode=RGB size=224x224 a...,aHR0cHM6Ly9idHBvY3ZoZHMuYmxvYi5jb3JlLndpbmRvd3...,./mnt/poeBlob1/output_images/aHR0cHM6Ly9idHBvY...,<azure.cosmosdb.table.models.EntityProperty ob...,"W/""datetime'2020-01-08T23%3A40%3A19.9774644Z'"""
1,PoeRecycle_test,1,2020-01-08 23:40:19.978465+00:00,[0.50558424 0.51385236 0.01591536 ... 0.396106...,<PIL.Image.Image image mode=RGB size=224x224 a...,aHR0cHM6Ly9idHBvY3ZoZHMuYmxvYi5jb3JlLndpbmRvd3...,./mnt/poeBlob1/output_images/aHR0cHM6Ly9idHBvY...,<azure.cosmosdb.table.models.EntityProperty ob...,"W/""datetime'2020-01-08T23%3A40%3A19.9784655Z'"""
2,PoeRecycle_test,10,2020-01-08 23:40:19.979465+00:00,[1.3110572 0.31182456 0.22548741 ... 0.013136...,<PIL.Image.Image image mode=RGB size=224x224 a...,aHR0cHM6Ly9idHBvY3ZoZHMuYmxvYi5jb3JlLndpbmRvd3...,./mnt/poeBlob1/output_images/aHR0cHM6Ly9idHBvY...,<azure.cosmosdb.table.models.EntityProperty ob...,"W/""datetime'2020-01-08T23%3A40%3A19.9794658Z'"""
3,PoeRecycle_test,100,2020-01-08 23:40:21.362437+00:00,[0.02647619 0.34645134 0. ... 0.248528...,<PIL.Image.Image image mode=RGB size=224x224 a...,aHR0cHM6Ly9idHBvY3ZoZHMuYmxvYi5jb3JlLndpbmRvd3...,./mnt/poeBlob1/output_images/aHR0cHM6Ly9idHBvY...,<azure.cosmosdb.table.models.EntityProperty ob...,"W/""datetime'2020-01-08T23%3A40%3A21.3624372Z'"""
4,PoeRecycle_test,1000,2020-01-08 23:40:27.639824+00:00,[0.11525676 0.89375764 0.00413675 ... 2.588224...,<PIL.Image.Image image mode=RGB size=224x224 a...,aHR0cHM6Ly9idHBvY3ZoZHMuYmxvYi5jb3JlLndpbmRvd3...,./mnt/poeBlob1/output_images/aHR0cHM6Ly9idHBvY...,<azure.cosmosdb.table.models.EntityProperty ob...,"W/""datetime'2020-01-08T23%3A40%3A27.6398245Z'"""


In [63]:
display(df.image[0])

'<PIL.Image.Image image mode=RGB size=224x224 at 0x7F63DB7849D0>'